<a href="https://colab.research.google.com/github/dominiquebuford/AIPI540-project1/blob/main/ResNet_no_freezing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Necessary Libraries and Set up Kaggle API

In [ ]:
#import necessary dependencies
import os
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import random_split
!pip install optuna
import optuna

In [ ]:
# set up access to Kaggle API
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json


Download Kaggle Dataset Into Current Directory

In [ ]:
# download dataset from Kaggle and unzip file to store dataset in colab
!kaggle datasets download -d itsahmad/indoor-scenes-cvpr-2019
!unzip indoor-scenes-cvpr-2019.zip

In [ ]:
images_dir = './indoorCVPR_09/Images/' #replace with location of your images

In [ ]:
def create_datasets_with_transformations(image_dir):
  transformations = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(degrees=10),
      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),  #normalize pixel values
  ])
  dataset = ImageFolder(image_dir, transform = transformations)
  train_ds, val_ds, test_ds = random_split(dataset, [.6, .2, .2])
  return train_ds, val_ds, test_ds, dataset.classes



Define Custom ResNet Model Class

In [ ]:
class CustomResNet(nn.Module):
  def __init__(self, fc_structure):
    super().__init__()
    self.model = models.resnet18(pretrained = True)
    for param in self.model.parameters():
      param.requires_grad = True
    self.model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    self.model.fc = fc_structure

  def forward(self,x):
    return self.model(x)


In [ ]:
def train_batch(x, y, model, opt, loss_fn):
    opt.zero_grad()
    model.train()
    logits = model(x.cuda())
    batch_loss = loss_fn(logits.squeeze(), y.cuda())
    batch_loss.backward()
    opt.step()
    opt.zero_grad()
    return batch_loss.item()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    logits = model(x.cuda())
    pred = logits.argmax(1).flatten().cpu()
    is_correct = (pred) == y
    return is_correct.numpy().tolist()


Try Different Structures for FC layer

In [ ]:
structure_1 = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, len(classes)),
    nn.Softmax(dim=1))

structure_2 = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, 100),
    nn.ReLU(),
    nn.Linear(100, len(classes)),
    nn.Softmax(dim =1))




Create Tensor Dataloaders

In [ ]:
train_ds, val_ds, test_ds, classes = create_datasets_with_transformations(images_dir)


In [ ]:
def get_data(train_ds, val_ds):
    trn_dl = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last = True)
    val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, drop_last = True)
    return trn_dl, val_dl

Define Optuna Objective

In [ ]:
def objective(trial):
    # Define hyperparameters to tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    epochs = trial.suggest_int('epochs', 20, 30)
    # Create the transfer learning model with the given hyperparameters
    model = CustomResNet(fc_structure = structure_2)
    model = model.to(device)

    # Set up optimizer, loss function, and other necessary components
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    train_ds, val_ds, test_ds, classes = create_datasets_with_transformations(images_dir)
    trn_dl, val_dl = get_data(train_ds, val_ds)

    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
      print(f'Epoch {epoch+1}/{epochs}\n')
      train_epoch_accuracies, val_epoch_accuracies = [], []

      for ix, batch in enumerate(iter(trn_dl)):
          x, y = batch
          train_batch(x, y, model, optimizer, loss_fn)

      for ix, batch in enumerate(iter(trn_dl)):
          x, y = batch
          is_correct = accuracy(x, y, model)
          train_epoch_accuracies.extend(is_correct)
      train_epoch_accuracy = np.mean(train_epoch_accuracies)

      for ix, batch in enumerate(iter(val_dl)):
          x, y = batch
          is_correct = accuracy(x, y, model)
          val_epoch_accuracies.extend(is_correct)
      val_epoch_accuracy = np.mean(val_epoch_accuracies)

      val_accuracies.append(val_epoch_accuracy)

    return max(train_accuracies)







In [ ]:
study = optuna.create_study(direction='maximize')  # or 'minimize' depending on your metric
study.optimize(objective, n_trials=1)

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

Rerun Model with Best Parameters Found

In [ ]:
#run final model with best parameters from optuna
best_params = trial.params

model = CustomResNet(fc_structure = structure_2)
model = model.to(device)

train_ds, val_ds, test_ds, classes = create_datasets_with_transformations(images_dir)
trn_dl, val_dl = get_data(train_ds, val_ds)

# Set up optimizer, loss function, and other necessary components
optimizer = optim.SGD(model.parameters(), lr=best_params['learning_rate'])
loss_fn = nn.CrossEntropyLoss()

train_accuracies, val_accuracies = [], []

for i in range(best_params['epochs'])
  for ix, batch in enumerate(iter(trn_dl)):
    x, y = batch
    train_batch(x, y, model, optimizer, loss_fn)

  train_epoch_accuracies, val_epoch_accuracies = [], []
  for ix, batch in enumerate(iter(trn_dl)):
    x, y = batch
    is_correct = accuracy(x, y, model)
    train_epoch_accuracies.extend(is_correct)
  train_epoch_accuracy = np.mean(train_epoch_accuracies)
  train_accuracies.append(train_epoch_accuracy)

  for ix, batch in enumerate(iter(val_dl)):
    x, y = batch
    is_correct = accuracy(x, y, model)
    val_epoch_accuracies.extend(is_correct)
  val_epoch_accuracy = np.mean(val_epoch_accuracies)
  val_accuracies.append(val_epoch_accuracy)



In [ ]:
epochs = np.arange(30)+1
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline
plt.plot(epochs, train_accuracies, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracies, 'r', label='Validation accuracy')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.title('Training and validation accuracy with ResNet18')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0.01,1)
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])
plt.legend()
plt.grid('off')
plt.show()